# SEC Data Collection
### Steps
1. Get urls for filings
2. Get company info and filter by marketcap
4. Download filings
5. Clean filings
6. Diff filings

In [3]:
# 1. Get urls for 2019 Q3 filings and more recent

import edgar
import os

data_dir = os.path.join(os.getcwd(), 'data/sec')
# os.mkdir(data_dir)
indices_dir = os.path.join(data_dir, 'indices')
# os.mkdir(indices_dir)
# edgar.download_index(indices_dir, 2019, False)

In [ ]:
"""
2. Get company info and filter by marketcap
    a. Get list of ticker symbols
    b. Join symbols with filings dataset on cik
    c. Get marketcap for each
    d. Get sector for each
    e. Filter by marketcap
""" 
# Reference code: https://github.com/PlatorSolutions/quarterly-earnings-machine-learning-algo/blob/master/download_raw_html.py
import pandas as pd
import os

filings_dir = os.path.join(data_dir, 'filings')
# os.mkdir(filings_dir)

df = pd.DataFrame()
for tsv in os.listdir(indices_dir):
    if '.ipynb_checkpoints' in tsv:
        continue
    path = os.path.join(indices_dir, tsv)
    df_ext = pd.read_csv(path, delimiter='|', names=['cik', 'name', 'type', 'date', 'text_uri', 'index_url'])
    df = df.append(df_ext)
    
df_10Q = df[df['type'] == '10-Q'].reset_index()
df_10Q['date'] = pd.to_datetime(df_10Q['date'], format='%Y-%m-%d')

In [ ]:
import requests
import numpy as np

def cik_to_ticker(cik):
    endpoint = f'https://mapping-api.herokuapp.com/cik/{cik}'
    res = requests.get(endpoint)
    if len(res.json()) < 1:
        return np.nan
    return res.json()[0]['ticker']

In [ ]:
def ticker_to_company_info(ticker):
    key_id = 'ALPACA or POLYGON key'
    endpoint = f'https://api.polygon.io/v1/meta/symbols/{ticker}/company?apiKey={key_id}'
    res = requests.get(endpoint)
    if res.status_code == 200 and 'json' in res.headers['Content-Type']:
        return res.json()
    else:
        return np.nan

In [ ]:
df_10Q['ticker'] = df_10Q['cik'].apply(lambda cik: cik_to_ticker(cik))

In [ ]:
df_10Q = df_10Q.dropna()
company_info_list = df_10Q['ticker'].apply(lambda ticker: ticker_to_company_info(ticker))

In [ ]:
bool_index = company_info_list.notna()
company_info_list = company_info_list.loc[bool_index]
df_10Q = df_10Q.loc[bool_index].reset_index()

In [ ]:
df_10Q['marketcap'] = company_info_list.apply(lambda info: info['marketcap'])
df_10Q['sector'] = company_info_list.apply(lambda info: info['sector'])
df_10Q = df_10Q.dropna()

In [ ]:
df_10Q['marketcap'].describe()

In [ ]:
df_10Q = df_10Q.loc[df_10Q['marketcap'] >= 1e9]

In [35]:
# 3. Download filings
import requests
from bs4 import BeautifulSoup

filing_dir = os.path.join(data_dir, 'raw_filings')
# os.mkdir(filing_dir)

sec_url = 'https://www.sec.gov/Archives'
min_change = 1
curr_change = 0
path_list = []
for i, filing in df_10Q.iterrows():
    if i > 0:
        curr_val = (i / len(df_10Q)) * 100
        curr_change += curr_val - (((i - 1) / len(df_10Q)) * 100)
        if curr_change >= min_change:
            print(f'{curr_val:.2f}% downloaded')
            curr_change = 0
    
    # Get page
    path = os.path.join(sec_url, filing['index_url'])
    res = requests.get(path)
    soup = BeautifulSoup(res.content, 'lxml')
    
    # Find download href
    files_table = soup.find('table', {'class': 'tableFile', 'summary': 'Document Format Files'})
    anchor_element = None
    for row in files_table.find_all('tr'):
        e = row.find_all('td')
        if len(e) >= 4 and e[3].text == filing['type']:
            anchor_element = e[2].find('a', href=True)
            break
    
    anchor = anchor_element['href']
    name = anchor_element.text
    acceptance_date = soup.find('div', attrs={'class': 'infoHead'}, text='Accepted').findNext('div', {'class': 'info'}).text

    if 'ix?' in anchor:
        anchor = '/' + '/'.join(anchor.split('/')[2:])
    
    # Download
    sec_base = 'https://www.sec.gov'
    download_res = requests.get(sec_base + anchor)
    
    # Save
    file_name = '_'.join([filing['name'].replace(' ', '').replace('/', ''), filing['type'], filing['date'].strftime('%Y-%m')]) + '.htm'
    file_path = os.path.join(filing_dir, file_name)
    with open(file_path, 'w') as file:
        file.write(download_res.text)
    
    path_list.append(file_path)

1.09% downloaded
2.19% downloaded
3.28% downloaded
4.38% downloaded
5.47% downloaded
6.57% downloaded
7.66% downloaded
8.76% downloaded
9.85% downloaded
10.94% downloaded
12.04% downloaded
13.13% downloaded
14.23% downloaded
15.32% downloaded
16.42% downloaded
17.51% downloaded
18.60% downloaded
19.70% downloaded
20.79% downloaded
21.89% downloaded
22.98% downloaded
24.08% downloaded
25.17% downloaded
26.27% downloaded
27.36% downloaded
28.45% downloaded
29.55% downloaded
30.64% downloaded
31.74% downloaded
32.83% downloaded
33.93% downloaded
35.02% downloaded
36.11% downloaded
37.21% downloaded
38.30% downloaded
39.40% downloaded
40.49% downloaded
41.59% downloaded
42.68% downloaded
43.78% downloaded
44.87% downloaded
45.96% downloaded
47.06% downloaded
48.15% downloaded
49.25% downloaded
50.34% downloaded
51.44% downloaded
52.53% downloaded
53.63% downloaded
54.72% downloaded
55.81% downloaded
56.91% downloaded
58.00% downloaded
59.10% downloaded
60.19% downloaded
61.29% downloaded
6

In [36]:
df_10Q['path'] = path_list

In [ ]:
df_10Q.to_csv(os.path.join(data_dir, 'df_10Q.csv'))

In [38]:
df_10Q = pd.read_csv(os.path.join(data_dir, 'df_10Q.csv'), parse_dates=['date']) 

In [39]:
# 5. Clean filings
import html
import multiprocessing as mp
import re
from functools import partial

def removeInnerLinks(soup):
    [i.extract() for i in soup.find_all('a', href=True) if len(i['href']) > 0 and i['href'][0] == '#']
    return soup

def remove_xbrli(soup):
    [x.extract() for x in soup.find_all(re.compile("^xbrli:"))]
    return soup

def removeNumericalTables(soup):

    def GetDigitPercentage(tablestring):
        if len(tablestring) > 0.0:
            numbers = sum([char.isdigit() for char in tablestring])
            length = len(tablestring)
            return numbers / length
        else:
            return 1

    def containsBgColor(table):
        for row in table.find_all('tr'):
            colored = 'background-color' in str(row) or 'bgcolor' in str(row)
            if colored:
                return True
        return False

    [x.extract() for x in soup.find_all('table') if containsBgColor(x)]

    [x.extract() for x in soup.find_all('table') if GetDigitPercentage(x.get_text()) > 0.15]

    return soup

def prep_text(text):
    soup = BeautifulSoup(html.unescape(re.sub(r'\s+', ' ', text)), "lxml")

    soup = remove_xbrli(soup)

    soup = removeInnerLinks(soup)

    soup = removeNumericalTables(soup)

    [x.unwrap() for x in soup.find_all(['span', 'font', 'b', 'i', 'u', 'strong'])]

    soup.smooth()

    text = soup.get_text(separator="\n", strip=True)

    stop_words = set(['HUNDRED', 'THOUSAND', 'MILLION', 'BILLION', 'TRILLION', 'DATE', 'ANNUAL', 'ANNUALLY', 'ANNUM', 'YEAR', 'YEARLY', 'QUARTER', 'QUARTERLY', 'QTR', 'MONTH', 'MONTHLY', 'WEEK', 'WEEKLY', 'DAY', 'DAILY', 'JANUARY', 'FEBRUARY', 'MARCH', 'APRIL', 'MAY', 'JUNE', 'JULY', 'AUGUST', 'SEPTEMBER', 'OCTOBER', 'NOVEMBER', 'DECEMBER', 'JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'SEPT', 'OCT', 'NOV', 'DEC', 'MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY', 'ONE', 'TWO', 'THREE', 'FOUR', 'FIVE', 'SIX', 'SEVEN', 'EIGHT', 'NINE', 'TEN', 'ELEVEN', 'TWELVE', 'THIRTEEN', 'FOURTEEN', 'FIFTEEN', 'SIXTEEN', 'SEVENTEEN', 'EIGHTEEN', 'NINETEEN', 'TWENTY', 'THIRTY', 'FORTY', 'FIFTY', 'SIXTY', 'SEVENTY', 'EIGHTY', 'NINETY', 'FIRST', 'SECOND', 'THIRD', 'FOURTH', 'FIFTH', 'SIXTH', 'SEVENTH', 'EIGHTH', 'NINTH', 'TENTH'])
    pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*', re.IGNORECASE)
    text = pattern.sub('', text)

    pattern = re.compile('\s[^a-zA-Z\s]+?(?=(\.*\s))')
    text = pattern.sub('', text)

    text = '\n'.join(
        filter(lambda line: len(line) > 0 and (sum(i.isalpha() for i in line) / len(line) > .5), text.splitlines()))

    return text


def prep_file(path):
    with open(path) as f:
        text = prep_text(f.read())
    return text

def saveBag(c, old_file_name):
    file_name = f'cleaned_{old_file_name.replace(".htm", ".txt")}'
    file_path = os.path.join(cleaned_filings_dir, file_name)
    with open(file_path, 'w') as file:
        file.write(c)

In [40]:
import os
from bs4 import BeautifulSoup

cleaned_filings_dir = os.path.join(data_dir, 'cleaned_filings')
# os.mkdir(cleaned_filings_dir)

pool = mp.Pool(mp.cpu_count())
for file in os.listdir(filing_dir):
    file_path = os.path.join(filing_dir, file)
    
    callback = partial(saveBag, old_file_name=file)
    pool.apply_async(prep_file, args=[file_path], callback=callback, error_callback=lambda x: print('Error: ', x))
    
pool.close()
pool.join()

Error:  [Errno 21] Is a directory: '/home/zion/Documents/cs5830/final_project/data/sec/raw_filings/.ipynb_checkpoints'


In [41]:
def load_text(path):
    with open(path, 'r') as file:
        return file.read()

In [57]:
df_10Q['text'] = df_10Q['path'].apply(lambda path: load_text(path))

In [56]:
# Remove some missed garbage tokens
from unidecode import unidecode
df_10Q['text'] = df_10Q['text'].apply(lambda text: unidecode(text))

In [58]:
# Compress
df_10Q.to_pickle('df_10Q.pkl')

### ToDo:
* possibly divide filings by section
* experiment with diffing filings (fuzzywuzzy)